In [21]:
'''
Secção com funções auxiliares - até conseguir por os módulos a funcionar...
'''

def validar_dna(seq):
    
    """
    Devolve a validade de uma sequência de ADN

    
    Parameters
    -------------
    seq : str
        Uma string que representa a sequência de ADN

    
    Returns
    -------------
    bool : True se for uma sequência válida ou False se for uma sequência inválida
    
    """
    if not bool(seq): return False
    
    seq = seq.upper()        
    seq = seq.replace(" ","")   

    valido_dna = True
    
    for base in seq:                  
        if base not in "ATCG":         
            valido_dna = False
    
    if valido_dna == False:
        return False
    else:
        return True

def aprimorar_seq(seq):
    """
    Devolve uma sequência de ADN só com letras maiúsculas e sem espaços em branco

    Parameters
    -------------
    seq : str
        Uma string que representa a sequência de ADN

    Returns
    -------------
    str
        sequência de ADN com as bases em maiúsculas e sem espaços em branco
    
    """
    return seq.upper().replace(" ","")
     

''' Função usada para formatar o conteúdo, que irá ajudar na formatação da matriz '''

def formatar(conteudo):

    if not isinstance(conteudo, (int,float,str)): 
        print('Inválido, tem de ser um número ou string.')
        return None

    if type(conteudo) is int:

        '''Se x for um int converte-o em str e alinha-o à direita (por causa do indicador de formatação '>')
  
        Se pretendessemos alinhar à esquerda o indicador de formatação seria '<'
    
        A parte 3d significa o número mínimo de dígitos de largura que queremos no alinhamento.''' 
    

        return f"{conteudo:>3d}"

    else:

        '''
        Caso não seja um int alinha na mesma à direita mas aqui o fator d já não se aplica uma vez que o format
        code d não se aplica a strs por exemplo.
        '''

        return f"{conteudo:>3}"
    
''' Função usada para imprimir a matriz corretamente formatada '''

def print_matrix(primeira_string : str, segunda_string : str, matriz_scores : list):
  '''
  S1 -> primeira string
  S2 -> segunda string
  M  -> matriz

  '''

  '''
  Começamos por construir e formatadar a nossa linha de colunas.

  Para isso convertemos a string 1, correspondente às colunas, em lista e iteramos elemento a elemento.

  Usamos operador * para fazer unpack da lista, carater a carater. Desta forma não imprime a lista como um todo. 
  Neste caso usamos definimos o parâmetro sep da função print, indicando 2 espaços em branco para espaçar melhor a nossa linha de colunas.
  
  '''
  
  print(*list(" " + primeira_string), sep = '   ')

  '''
  Formatamos o conteúdo da nossa coluna de linhas e respetivo conteúdo das linhas.

  Para isto usamos um ciclo for que irá iterar por cada elemento do emparelhamento da nossa segunda string e a matriz de scores obtida.

  '''

  for linha, conteudo_linha in zip(segunda_string, matriz_scores):
    #print('linha', linha)
    conteudo_formatado = [formatar(conteudo) for conteudo in conteudo_linha]
    print(linha, *conteudo_formatado)

  print()       # Adicionamos um alinha em branco para separar do conteúdo seguinte.

'''
Função auxiliar que reconstroi as sequências devidamente alinhadas.

'''

'''
Função que encontra as coordenadas do valor mais alto da matrix score.

'''
def max_score_loc(scr):
    ms = scr[0][0]
    mi,mj = 0,0
    for linha in range(len(scr)): # itera por cada linha
    
        for coluna in range(len(scr[linha])): # itera por cada item de cada linha 
            if scr[linha][coluna] > ms:
                ms = scr[linha][coluna] # regista o maior
                mi,mj = linha,coluna # regista as coordenadas
    
    return mi,mj

'''

Função que reconstroi a sequência alinhada em função do algoritmo selecionado (NW ou SW)

'''

def reconstroi(string_1, string_2, score, matriz_traceback, algoritmo):
  '''
  Parameteres
  -----------

  string_1 : str
    primeira string para alinhamento

  string_1 : str
    segunda string para alinhamento

  matriz_traceback : list
    lista de listas correspondente a uma matriz com direções para percorrer as strings
    e reconstruir os alinhamentos

  Returns
  -------

  tuple
    devolve um tuple com as matrizes alinhadas
  
  '''
  
  # Iniciamos as variáveis-resultado
  string_1_alinhada = ''
  string_2_alinhada = ''
  
  if algoritmo == 'nw':
    # Criamos os contador para iterar, que nos devolverá o último elemento de cada string
    colunas = len(string_1) - 1
    linhas  = len(string_2) - 1
    

    # Itera pela matriz traceback e em função da direção calculada determina a posição e o elemento a atribuir.
    
    while matriz_traceback[linhas][colunas] != 0:
      # print(linhas, colunas, string_2[linhas], string_1[colunas], matriz_traceback[linhas][colunas])

      # Se na matriz andarmos na diagonal significa que os elementos são iguais, então saltam para o alinhamento
      if matriz_traceback[linhas][colunas] == 'D':
        string_1_alinhada = string_1[colunas] + string_1_alinhada  # De notar que adicionamos sempre a string alinhada no fim 
        string_2_alinhada = string_2[linhas]  + string_2_alinhada  # pois estamos a construí-la de trás para a frente

        linhas  -= 1                                               # Subtraímos 1 a linhas e colunas para avançar para percorrer
        colunas -= 1                                               # mais um nível na nossa matriz traceback

      # Se na matriz andarmos para cima, significa que não são iguais e conservamos o elemento da segunda string
      elif matriz_traceback[linhas][colunas] == 'C':
        string_1_alinhada = '-'              + string_1_alinhada
        string_2_alinhada = string_2[linhas] + string_2_alinhada

        linhas  -= 1                                              # Como andamos para cima só avançamos nas linhas

      # Se na matriz andarmos para a esquerda, significa que não são iguais e conservamos o elemento da primeira string
      elif matriz_traceback[linhas][colunas] == 'E':
        string_1_alinhada = string_1[colunas] + string_1_alinhada
        string_2_alinhada = '-'               + string_2_alinhada

        colunas -= 1                                               # Como andamos para a esquerda só avançamos nas colunas

  elif algoritmo == 'sw':
    coord_string_1, coord_string_2 = max_score_loc(score)

    # i = linha (s2)
    # j = coluna (s1)

#    s1 = '-' + s1
#    s2 = '-' + s2
#    print(coord_string_2,coord_string_1,s1[coord_string_2],s2[coord_string_1])
    while coord_string_1 > 0 and coord_string_2 > 0:

        if matriz_traceback[coord_string_1][coord_string_2] == 'D':
            string_1_alinhada = string_1[coord_string_2] + string_1_alinhada
            string_2_alinhada = string_2[coord_string_1] + string_2_alinhada

            coord_string_1 -= 1
            coord_string_2 -= 1

        elif matriz_traceback[coord_string_1][coord_string_2] == 'C':
            string_1_alinhada = '-' + string_1_alinhada
            string_2_alinhada = string_2[coord_string_1] + string_2_alinhada

            coord_string_1 -= 1
            coord_string_2 -= 1

        elif matriz_traceback[coord_string_1][coord_string_2] == 'E':
            string_1_alinhada = string_1[coord_string_2] + string_1_alinhada
            string_2_alinhada = '-' + string_2_alinhada

            coord_string_1 -= 1
            coord_string_2 -= 1
        
        elif matriz_traceback[coord_string_1][coord_string_2] == 0:
            string_1_alinhada = '-' + string_1_alinhada
            string_2_alinhada = '-' + string_2_alinhada

            coord_string_1 -= 1
            coord_string_2 -= 1
  
  else: raise ValueError("Algoritmo inválido - escolher entre Needleman-Wunch (NW) ou Smith-Waterman (SW)")
     
  print()
  print(string_1_alinhada, string_2_alinhada, sep = "\n")        # Imprimimos as nossas strings alinhadas em linhas separadas

  return (string_1_alinhada, string_2_alinhada)                  # Obtemos as strings alinhadas reconstruídas para uso futuro

''' Função que substitui os elementos das strings e devolve o seu score comparativo '''

def score_subst(ele_string_1, ele_string_2, algoritmo, g = 0):
  '''Função que compara e substitui um elemento de cada uma das strings.

  Se a comparação for feita com um gap, devolve o valor correspondente a um gap'''

  if '-' in ele_string_1 + ele_string_2: return g
  
  ''' 
  É então comparado se os elementos são iguais. Se forem iguais, obtemos um score positivo
  
  Caso contrário obtemos uma penalidade/valor negativo

  A função tem um comportamento diferente em função do algoritmo de alinhamento.

  Menos permissiva para Smith Waterman para garantir o alinhamento local e mais flexível para Needleman Wunch 
  para abrir mais opções de alinhamento global 
  
  '''

  if algoritmo.upper() == 'NW':

    if ele_string_1 == ele_string_2:
      return 2
    else:
      return -1

  elif algoritmo.upper() == 'SW':
    
      if ele_string_1 == ele_string_2:
        return 1
      else:
        return -1
      



def tipo_seq(seq):
    """
    Função que a partir de uma sequência, classifica-a em DNA, RNA ou SEQUÊNCIA AMINÁCIDOS.

    Parâmetro
    ---------
    seq : str 
        A sequência a ser classificada.

    Retorna
    -------
    str
        A classificação da sequência.
        
    """

    if not bool(seq):
        raise ValueError("É uma sequência inválida")

    seq = aprimorar_seq(seq)
    
    if validar_dna(seq):
        return "DNA"
        
    if len([c for c in seq if c not in "ACGU"]) == 0:
        return "RNA"
        
    elif len([c for c in seq if c not in "ABCDEFGHIKLMNPQRSTVWYZ_"]) == 0:
        return "SEQUÊNCIA AMINOÁCIDOS"
    
    else:
        raise ValueError("É uma sequência inválida")
    




def complemento_inverso(sequencia : str) -> str:
    '''
    Função que itera sobre uma sequência de DNA e devolve
    o complemento reverso de cada nucleotido.

    Parametros 
    ----------

    sequencia : str
        sequência de DNA válida

    Retorna
    -------
    str
        sequência de nucléotidos complementar à cadeia fornecida

    Levanta
    -------

    ValueError
        caso não se trate de uma sequênica de DNA válida
        
    '''
    assert sequencia != '' or sequencia != ' '
    #assert validar_dna(sequencia)
    

    complementar = '' # inicializamos a cadeia complementar

    if tipo_seq(sequencia) == 'DNA':
        for base in sequencia.upper():
            if base == 'A':
                complementar += 'T'
            elif base == 'T':
                complementar += 'A'
            elif base == 'C':
                complementar += 'G'
            elif base == 'G':
                complementar += 'C'
    
    elif tipo_seq(sequencia) == 'RNA':
        for base in sequencia.upper():
            if base == 'A':
                complementar += 'U'
            elif base == 'U':
                complementar += 'A'
            elif base == 'C':
                complementar += 'G'
            elif base == 'G':
                complementar += 'C'
    
    else:
        raise ValueError('É uma sequência inválida')

    return complementar[::-1]





def get_orfs(seq):
    """
    Função que a partir de uma sequência de DNA ou RNA origina diferentes tipos de ORFS.

    Parâmetro
    --------
    seq : str
        Sequência de DNA ou RNA.

    Returns
    -------
    lista
        Lista das ORFs geradas a partir da sequência.
    
    """
    seq_comp_inv = complemento_inverso(seq)
    lista_orfs = [
            seq[0:], seq[1:], seq[2:],
            seq_comp_inv[0:], seq_comp_inv[1:], seq_comp_inv[2:]
        ]
    return lista_orfs
    



In [7]:


'''

Função de alinhamento que conjunta as subfunções e permite selecionar o algoritmo pretendido.

'''

def alinhar_sequencias(string_1, string_2, algoritmo, score_space = -4):
  
  if algoritmo == 'nw':

    needleman_wunch(string_1, string_2, score_subst, score_space)

  elif algoritmo == 'sw':

    smith_waterman(string_1, string_2, score_subst, score_space)

  else: raise ValueError("Algoritmo inválido - escolher entre Needleman-Wunch (NW) ou Smith-Waterman (SW)")


    



'''

Sub-funções da função alinhamento().

Algoritmo de Needleman-Wunch

'''


def needleman_wunch(string_1 : str, string_2 : str, score_subst : int, score_space : int = -4) -> tuple: 
  '''
  Parameters
  ----------
  string_1 : str
    primeira string

  string_2 : str 
    segunda string
  
  score_space : int
    valor penalidade atribuido a espaços vazios
  
  score_subst : int
    função que recebe dois caracteres e devolve o score da substituição de um pelo outro

  Returns
  -------

  tuple
    devolve um tuplo com o score do melhor alinhamento possível e o respetivo alinhamento
    
  '''
  # Garantimos que recebemos uma sequência. Este algoritmo não é só para DNA / aminoácidos por isso não usamos o validar_dna

  assert type(string_1) and type(string_2) == str

  # Acrescentamos os gaps para iniciar a matriz corretamente
  string_1 = '-' + string_1
  string_2 = '-' + string_2

  # Definimos o número de linhas e colunas da nossa matriz, posicionando as nossas strings
  # corretamente
  ncols = len(string_1)
  nlins = len(string_2)

  # Inicializar toda a matriz a zeros
  score = [[0 for gap in range(ncols)] for gap in range(nlins)]

  matriz_traceback = [[0 for gap in range(ncols)] for gap in range(nlins)]

  # Inicializar a primeira linha
  # Atribuí-se o valor de penalidade porque a primeira linha corresponde apenas a alinhamento espaços vazios
  # (ver output para visualizar)
  score[0] = [posicao_coluna * score_space for posicao_coluna, elemento in enumerate(string_1)]

  # Como estamos na primeira linha, estamos a alinhar apenas com espaços vazios
  # pelo que a matriz traceback indicará que devemos sempre caminhar à esquerda
  # (ver output para visualizar)

  matriz_traceback[0] = [0 if posicao_coluna == 0 else 'E' for posicao_coluna, elemento in enumerate(string_1)]


  # Inicializar a primeira coluna
  
  for posicao_linha, elemento in enumerate(string_2):
  
    # A lógica de preenchimento é semelhante à das linhas, mudando apenas 
    # a direção, que será vertical

    # Iteramos sobre o indice correspondente à posição do elemento na linha, indice 0 que corresponde
    # a primeira coluna 
    score[posicao_linha][0] = posicao_linha * score_space
  
    # Conforme mencionado acima, mesma lógica mas na vertical
    matriz_traceback[posicao_linha][0] = 0 if posicao_linha == 0 else 'C'


  # Iniciamos o primeiro ciclo for, que emparelha a segunda string com o a matriz score, correndo
  # linha a linha
  for posicao_linha, (elemento_string_2, linha) in enumerate(zip(string_2, score)):

    # No segundo ciclo for começamos a fazer a comparação dos elementos das strings, coluna a coluna

    for posicao_coluna, (elemento_string_1, valor_score) in enumerate(zip(string_1, linha)):

      
      if posicao_linha > 0 and posicao_coluna > 0: # Parte do 0 pois ignora a posição 0 são gaps

        # Compara os elementos ao longo de toda a matriz atribuindo os scores.

        # Compara diagonal
        diag = score[posicao_linha - 1][posicao_coluna - 1] + score_subst(elemento_string_1, elemento_string_2, score_space)
        
        # Compara à esquerda
        esq  = score[posicao_linha    ][posicao_coluna - 1] + score_space

        # Compara acima
        cima = score[posicao_linha - 1][posicao_coluna    ] + score_space

        # Agrupa os resultados, determina o maior score e atribui uma das direções.
        # usa-se uma string de referência para facilitar a atribuição / ser mais eficiente

        escolhas = [diag, esq, cima]
        direcoes = "DEC"

        # Encontramos o valor maior que corresponderá ao melhor score
        # que levará ao alinhamento ótimo

        valor = max(*escolhas)
        
        # Populamos a matriz score
        score[posicao_linha][posicao_coluna] = valor

        # Convertemos em direção e populamos a matriz traceback
        matriz_traceback[posicao_linha][posicao_coluna] = direcoes[escolhas.index(valor)]

        

  # Recorremos à função print_matrix() para imprimir as nossas matrizes score e traceback

  print_matrix(string_1, string_2, score)
  print_matrix(string_1, string_2, matriz_traceback)

  '''
  Devolvemos o último valor da matriz score (o ponto de partida para fazer o traceback)
  acrescentamos também as sequências reconstruídas partindo da matriz de traceback e usando
  a função reconstroi
  '''

  return score[-1][-1], reconstroi(string_1, string_2, score, matriz_traceback, 'nw')



'''

Algoritmo de Smith-Waterman

'''

def smith_waterman(string_1 : str, string_2 : str, score_subst : int, score_space : int = -4) -> tuple: 
  '''
  Parameters
  ----------
  string_1 : str
    primeira string

  string_2 : str 
    segunda string
  
  score_space : int
    valor penalidade atribuido a espaços vazios
  
  score_subst : int
    função que recebe dois caracteres e devolve o score da substituição de um pelo outro

  Returns
  -------

  tuple
    devolve um tuplo com o score do melhor alinhamento possível e o respetivo alinhamento
    
  '''
  # Garantimos que recebemos uma sequência. Este algoritmo não é só para DNA / aminoácidos por isso não usamos o validar_dna

  assert type(string_1) and type(string_2) == str

  # Acrescentamos os gaps para iniciar a matriz corretamente
  string_1 = '-' + string_1
  string_2 = '-' + string_2

  # Definimos o número de linhas e colunas da nossa matriz, posicionando as nossas strings
  # corretamente
  ncols = len(string_1)
  nlins = len(string_2)

  # Inicializar toda a matriz a zeros
  score = [[0 for gap in range(ncols)] for gap in range(nlins)]
  max_score = max([val for linha in score for val in linha])

  matriz_traceback = [[0 for gap in range(ncols)] for gap in range(nlins)]



  # Iniciamos o primeiro ciclo for, que emparelha a segunda string com o a matriz score, correndo
  # linha a linha
  for posicao_linha, (elemento_string_2, linha) in enumerate(zip(string_2, score)):
    #print('#####linha',posicao_linha,'#####') 
    # No segundo ciclo for começamos a fazer a comparação dos elementos das strings, coluna a coluna

    for posicao_coluna, (elemento_string_1, valor_score) in enumerate(zip(string_1, linha)):
      #print('####coluna',posicao_coluna,'####')
      
      if posicao_linha > 0 and posicao_coluna > 0: # Parte do 0 pois ignora a posição 0 são gaps

        # Compara os elementos ao longo de toda a matriz atribuindo os scores.
        
        # Compara diagonal
        diag = score[posicao_linha - 1][posicao_coluna - 1] + score_subst(elemento_string_1, elemento_string_2)
        #print('diag',diag)
        #print('e1,e2',(elemento_string_1,elemento_string_2))
        
        # Compara à esquerda
        esq  = score[posicao_linha    ][posicao_coluna - 1] + score_space
        #print('esq',esq,'score-space',score_space)

        

        # Compara acima
        cima = score[posicao_linha - 1][posicao_coluna    ] + score_space
        #print('cima',cima)
        

        # Agrupa os resultados, determina o maior score e atribui uma das direções.
        # usa-se uma string de referência para facilitar a atribuição / ser mais eficiente

        escolhas = [diag, esq, cima, 0]
        direcoes = "DEC"

        # Encontramos o valor maior que corresponderá ao melhor score
        # que levará ao alinhamento ótimo

        valor = max(*escolhas)
        #print('valor=',valor)
        
        # Populamos a matriz score
        score[posicao_linha][posicao_coluna] = valor


        # Convertemos em direção e populamos a matriz traceback
        if valor != 0: matriz_traceback[posicao_linha][posicao_coluna] = direcoes[escolhas.index(valor)] 
      
        # else: matriz_traceback[posicao_linha][posicao_coluna] = 0

     

  # Recorremos à função print_matrix() para imprimir as nossas matrizes score e traceback

  print_matrix(string_1, string_2, score)
  print_matrix(string_1, string_2, matriz_traceback)

  '''
  Devolvemos o último valor da matriz score (o ponto de partida para fazer o traceback)
  acrescentamos também as sequências reconstruídas partindo da matriz de traceback e usando
  a função reconstroi
  '''
  # Smith Waterman parte do valor mais alto para a reconstrução, por isso o valor de partida tem de ser o maior da matriz
  # Para isso é preciso "espalmar" a matriz para encontrar o valor mais alto

  max_score = max([val for linha in score for val in linha])
  # print('pontuacao-maxima=',max_score)
  return max_score, reconstroi(string_1, string_2, score, matriz_traceback, 'sw')


# Alinhamento de sequências

> De modo a manter este notebook legível, foram utilizadas as funções auxiliares presentes no módulo [auxiliares.py](.\auxiliares.py)
> Estas funções foram devidamente desenvolvidas, documentadas e testadas no notebook (acrescentar aqui)

O alinhamento de sequências é uma ferramenta útil em vários contextos, como é o contexto do processamento de linguagem natural e previsão de escrita (i.e. autocorrect). No contexto da bioinformática, o alinhamento de sequências assume um papel preponderante para a identificação de sequências similares ou domínios conservados (seja em contexto de genómica seja em contexto de proteómica).

Permite-nos fazer assunções e correlações entre sequência, estrutura e função.

Eixstem um conjunto de abordagens a esta temática, praticamente todas envolvendo à comparação das sequências numa matrix e ir comparando elemento a elemento atribuindo valores a ocorrências idênticas (match), não-ocorrências (mismatch) e em certos cenários espaços vazios (gaps).

Em programação este problema pode ser resolvido utilizando técnicas de programação dinâmica que, apesar de eficazes, são muito pouco eficientes e custosas em termos de recursos da máquina.

Surgem então dois algoritmos excelentes para resolver esta problemática, sendo eles:

- O algoritom de Needleman-Wunch - utilizado em contexto de alinhamento global
- Smith-Waterman - como uma implementação do algoritmo Needleman-Wucnh mas com o acréscimo de mais uma camada, que substitui todos os valores negativos por 0, sendo um excelente algoritmo para alinhamento local / refinado.

Apresentamos abaixo as explicações descritivas e prescritivas para estes algoritmos bem como a sua implementação em Python.

## Demonstração 


In [8]:

# Sequências de teste
# Assume-se uma penalidade de espaço vazio de -4 - valor passado por defeito às nossas funções

s1 = 'ATTTCTCGCCTC'
s2 = 'ATC'

alinhar_sequencias(s1,s2,'nw')
print()
alinhar_sequencias(s1,s2,'sw')



    -   A   T   T   T   C   T   C   G   C   C   T   C
-   0  -4  -8 -12 -16 -20 -24 -28 -32 -36 -40 -44 -48
A  -4   2  -2  -6 -10 -14 -18 -22 -26 -30 -34 -38 -42
T  -8  -2   4   0  -4  -8 -12 -16 -20 -24 -28 -32 -36
C -12  -6   0   3  -1  -2  -6 -10 -14 -18 -22 -26 -30

    -   A   T   T   T   C   T   C   G   C   C   T   C
-   0   E   E   E   E   E   E   E   E   E   E   E   E
A   C   D   E   E   E   E   E   E   E   E   E   E   E
T   C   C   D   D   D   E   D   E   E   E   E   D   E
C   C   C   C   D   D   D   E   D   E   D   D   E   D


ATTTCTCGCCTC
A---------TC

    -   A   T   T   T   C   T   C   G   C   C   T   C
-   0   0   0   0   0   0   0   0   0   0   0   0   0
A   0   2   0   0   0   0   0   0   0   0   0   0   0
T   0   0   4   2   2   0   2   0   0   0   0   2   0
C   0   0   0   3   1   4   0   4   0   2   2   0   4

    -   A   T   T   T   C   T   C   G   C   C   T   C
-   0   0   0   0   0   0   0   0   0   0   0   0   0
A   0   D   0   0   0   0   0   0   0   0   0   0  

## Anatomia dos algoritmos de alinhamento

### Needleman-Wunch

Blabla o que é, o que faz.
Uma imagem talvez.

Código abaixo:

In [26]:
def needleman_wunch(string_1 : str, string_2 : str, score_subst : int, score_space : int = -4) -> tuple: 
  '''
  Parameters
  ----------
  string_1 : str
    primeira string

  string_2 : str 
    segunda string
  
  score_space : int
    valor penalidade atribuido a espaços vazios
  
  score_subst : int
    função que recebe dois caracteres e devolve o score da substituição de um pelo outro

  Returns
  -------

  tuple
    devolve um tuplo com o score do melhor alinhamento possível e o respetivo alinhamento
    
  '''
  # Garantimos que recebemos uma sequência. Este algoritmo não é só para DNA / aminoácidos por isso não usamos o validar_dna

  assert type(string_1) and type(string_2) == str

  # Acrescentamos os gaps para iniciar a matriz corretamente
  string_1 = '-' + string_1
  string_2 = '-' + string_2

  # Definimos o número de linhas e colunas da nossa matriz, posicionando as nossas strings
  # corretamente
  ncols = len(string_1)
  nlins = len(string_2)

  # Inicializar toda a matriz a zeros
  score = [[0 for gap in range(ncols)] for gap in range(nlins)]

  matriz_traceback = [[0 for gap in range(ncols)] for gap in range(nlins)]

  # Inicializar a primeira linha
  # Atribuí-se o valor de penalidade porque a primeira linha corresponde apenas a alinhamento espaços vazios
  # (ver output para visualizar)
  score[0] = [posicao_coluna * score_space for posicao_coluna, elemento in enumerate(string_1)]

  # Como estamos na primeira linha, estamos a alinhar apenas com espaços vazios
  # pelo que a matriz traceback indicará que devemos sempre caminhar à esquerda
  # (ver output para visualizar)

  matriz_traceback[0] = [0 if posicao_coluna == 0 else 'E' for posicao_coluna, elemento in enumerate(string_1)]


  # Inicializar a primeira coluna
  
  for posicao_linha, elemento in enumerate(string_2):
  
    # A lógica de preenchimento é semelhante à das linhas, mudando apenas 
    # a direção, que será vertical

    # Iteramos sobre o indice correspondente à posição do elemento na linha, indice 0 que corresponde
    # a primeira coluna 
    score[posicao_linha][0] = posicao_linha * score_space
  
    # Conforme mencionado acima, mesma lógica mas na vertical
    matriz_traceback[posicao_linha][0] = 0 if posicao_linha == 0 else 'C'


  # Iniciamos o primeiro ciclo for, que emparelha a segunda string com o a matriz score, correndo
  # linha a linha
  for posicao_linha, (elemento_string_2, linha) in enumerate(zip(string_2, score)):

    # No segundo ciclo for começamos a fazer a comparação dos elementos das strings, coluna a coluna

    for posicao_coluna, (elemento_string_1, valor_score) in enumerate(zip(string_1, linha)):

      
      if posicao_linha > 0 and posicao_coluna > 0: # Parte do 0 pois ignora a posição 0 são gaps

        # Compara os elementos ao longo de toda a matriz atribuindo os scores.

        # Compara diagonal
        diag = score[posicao_linha - 1][posicao_coluna - 1] + score_subst(elemento_string_1, elemento_string_2, 'nw', score_space)
        
        # Compara à esquerda
        esq  = score[posicao_linha    ][posicao_coluna - 1] + score_space

        # Compara acima
        cima = score[posicao_linha - 1][posicao_coluna    ] + score_space

        # Agrupa os resultados, determina o maior score e atribui uma das direções.
        # usa-se uma string de referência para facilitar a atribuição / ser mais eficiente

        escolhas = [diag, esq, cima]
        direcoes = "DEC"

        # Encontramos o valor maior que corresponderá ao melhor score
        # que levará ao alinhamento ótimo

        valor = max(*escolhas)
        
        # Populamos a matriz score
        score[posicao_linha][posicao_coluna] = valor

        # Convertemos em direção e populamos a matriz traceback
        matriz_traceback[posicao_linha][posicao_coluna] = direcoes[escolhas.index(valor)]

        

  # Recorremos à função print_matrix() para imprimir as nossas matrizes score e traceback

  print_matrix(string_1, string_2, score)
  print_matrix(string_1, string_2, matriz_traceback)

  '''
  Devolvemos o último valor da matriz score (o ponto de partida para fazer o traceback)
  acrescentamos também as sequências reconstruídas partindo da matriz de traceback e usando
  a função reconstroi
  '''

  return score[-1][-1], reconstroi(string_1, string_2, score, matriz_traceback, 'nw')



In [27]:
# Testes de alinhamento

s1 = 'ATCC'
s2 = 'ACG'
needleman_wunch(s1,s2,score_subst)

    -   A   T   C   C
-   0  -4  -8 -12 -16
A  -4   2  -2  -6 -10
C  -8  -2   1   0  -4
G -12  -6  -3   0  -1

    -   A   T   C   C
-   0   E   E   E   E
A   C   D   E   E   E
C   C   C   D   D   D
G   C   C   D   D   D


ATCC
A-CG


(-1, ('ATCC', 'A-CG'))

In [ ]:
# Testes de unidade

### Smith Waterman

Blabla o que é o que faz

Código abaixo


In [28]:
def smith_waterman(string_1 : str, string_2 : str, score_subst : int, score_space : int = -4) -> tuple: 
  '''
  Parameters
  ----------
  string_1 : str
    primeira string

  string_2 : str 
    segunda string
  
  score_space : int
    valor penalidade atribuido a espaços vazios
  
  score_subst : int
    função que recebe dois caracteres e devolve o score da substituição de um pelo outro

  Returns
  -------

  tuple
    devolve um tuplo com o score do melhor alinhamento possível e o respetivo alinhamento
    
  '''
  # Garantimos que recebemos uma sequência. Este algoritmo não é só para DNA / aminoácidos por isso não usamos o validar_dna

  assert type(string_1) and type(string_2) == str

  # Acrescentamos os gaps para iniciar a matriz corretamente
  string_1 = '-' + string_1
  string_2 = '-' + string_2

  # Definimos o número de linhas e colunas da nossa matriz, posicionando as nossas strings
  # corretamente
  ncols = len(string_1)
  nlins = len(string_2)

  # Inicializar toda a matriz a zeros
  score = [[0 for gap in range(ncols)] for gap in range(nlins)]
  max_score = max([val for linha in score for val in linha])

  matriz_traceback = [[0 for gap in range(ncols)] for gap in range(nlins)]



  # Iniciamos o primeiro ciclo for, que emparelha a segunda string com o a matriz score, correndo
  # linha a linha
  for posicao_linha, (elemento_string_2, linha) in enumerate(zip(string_2, score)):
    #print('#####linha',posicao_linha,'#####') 
    # No segundo ciclo for começamos a fazer a comparação dos elementos das strings, coluna a coluna

    for posicao_coluna, (elemento_string_1, valor_score) in enumerate(zip(string_1, linha)):
      #print('####coluna',posicao_coluna,'####')
      
      if posicao_linha > 0 and posicao_coluna > 0: # Parte do 0 pois ignora a posição 0 são gaps

        # Compara os elementos ao longo de toda a matriz atribuindo os scores.
        
        # Compara diagonal
        diag = score[posicao_linha - 1][posicao_coluna - 1] + score_subst(elemento_string_1, elemento_string_2, 'sw')
        #print('diag',diag)
        #print('e1,e2',(elemento_string_1,elemento_string_2))
        
        # Compara à esquerda
        esq  = score[posicao_linha    ][posicao_coluna - 1] + score_space
        #print('esq',esq,'score-space',score_space)

        

        # Compara acima
        cima = score[posicao_linha - 1][posicao_coluna    ] + score_space
        #print('cima',cima)
        

        # Agrupa os resultados, determina o maior score e atribui uma das direções.
        # usa-se uma string de referência para facilitar a atribuição / ser mais eficiente

        escolhas = [diag, esq, cima, 0]
        direcoes = "DEC"

        # Encontramos o valor maior que corresponderá ao melhor score
        # que levará ao alinhamento ótimo

        valor = max(*escolhas)
        #print('valor=',valor)
        
        # Populamos a matriz score
        score[posicao_linha][posicao_coluna] = valor


        # Convertemos em direção e populamos a matriz traceback
        if valor != 0: matriz_traceback[posicao_linha][posicao_coluna] = direcoes[escolhas.index(valor)] 
      
        # else: matriz_traceback[posicao_linha][posicao_coluna] = 0

     

  # Recorremos à função print_matrix() para imprimir as nossas matrizes score e traceback

  print_matrix(string_1, string_2, score)
  print_matrix(string_1, string_2, matriz_traceback)

  '''
  Devolvemos o último valor da matriz score (o ponto de partida para fazer o traceback)
  acrescentamos também as sequências reconstruídas partindo da matriz de traceback e usando
  a função reconstroi
  '''
  # Smith Waterman parte do valor mais alto para a reconstrução, por isso o valor de partida tem de ser o maior da matriz
  # Para isso é preciso "espalmar" a matriz para encontrar o valor mais alto

  max_score = max([val for linha in score for val in linha])
  # print('pontuacao-maxima=',max_score)
  return max_score, reconstroi(string_1, string_2, score, matriz_traceback, 'sw')




In [30]:
s1 = 'ACTGTGCGTCA'
s2 = 'ATC'

s1 = 'ACCGTC'
s2 = 'ATCGCC'


s1 = 'GATCGATCG'
s2 = 'AAGCTAAGC'
smith_waterman(s1,s2, score_subst)


    -   G   A   T   C   G   A   T   C   G
-   0   0   0   0   0   0   0   0   0   0
A   0   0   1   0   0   0   1   0   0   0
A   0   0   1   0   0   0   1   0   0   0
G   0   1   0   0   0   1   0   0   0   1
C   0   0   0   0   1   0   0   0   1   0
T   0   0   0   1   0   0   0   1   0   0
A   0   0   1   0   0   0   1   0   0   0
A   0   0   1   0   0   0   1   0   0   0
G   0   1   0   0   0   1   0   0   0   1
C   0   0   0   0   1   0   0   0   1   0

    -   G   A   T   C   G   A   T   C   G
-   0   0   0   0   0   0   0   0   0   0
A   0   0   D   0   0   0   D   0   0   0
A   0   0   D   0   0   0   D   0   0   0
G   0   D   0   0   0   D   0   0   0   D
C   0   0   0   0   D   0   0   0   D   0
T   0   0   0   D   0   0   0   D   0   0
A   0   0   D   0   0   0   D   0   0   0
A   0   0   D   0   0   0   D   0   0   0
G   0   D   0   0   0   D   0   0   0   D
C   0   0   0   0   D   0   0   0   D   0


A
A


(1, ('A', 'A'))

In [ ]:
# Testes de unidade
#  Caso 1
AGCTAGCAT
AGCTCGCAT

# Caso 2
Perfect match expectável
ATCGATCG
ATCGATCG

# Caso 3
Sem alinhamento possível
GATCGATCG
AAGCTAAGC

# Caso 4

# Caso 5